In [2]:
! pip install pyspark
! pip install findspark
! pip install folium
! pip install geopy

In [3]:
import os
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, min, max, countDistinct, desc, explode, split, col, isnull, sum, isnan, when, count
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from scipy.stats import kurtosis, skew
import math
import folium
from geopy.geocoders import Nominatim

In [4]:
findspark.init()

In [5]:
spark_url = 'local'
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark SQL')\
        .getOrCreate()
spark.conf.set("spark.sql.csv.parser.multiLine", "true")

In [6]:
path = 'bangkok_traffy.csv'
df = spark.read.option("multiLine", "true").csv(path, header=True, inferSchema=True)
df.show(5)

+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------+--------------------+--------------------+--------------+----+------------+--------------------+
|  ticket_id|               type|        organization|             comment|               photo|         photo_after|            coords|             address|subdistrict|district|            province|           timestamp|         state|star|count_reopen|       last_activity|
+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------+--------------------+--------------------+--------------+----+------------+--------------------+
|2021-9LHDM6|                 {}|                null|            ไม่มีภาพ|https://storage.g...|                null|100.48661,13.79386|1867 จรัญสนิทวงศ์...|    บางพลัด| บางพล

In [7]:
null_counts = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
null_counts.show()

+---------+----+------------+-------+-----+-----------+------+-------+-----------+--------+--------+---------+-----+------+------------+-------------+
|ticket_id|type|organization|comment|photo|photo_after|coords|address|subdistrict|district|province|timestamp|state|  star|count_reopen|last_activity|
+---------+----+------------+-------+-----+-----------+------+-------+-----------+--------+--------+---------+-----+------+------------+-------------+
|     2413|1550|        2640|   3911| 2089|      85624|  2019|   4433|       2089|    2092|    2393|     2233| 2026|164087|      120042|         2558|
+---------+----+------------+-------+-----+-----------+------+-------+-----------+--------+--------+---------+-----+------+------------+-------------+



In [8]:
# Filter the DataFrame to include only rows with null values in the "column_name" column
ticket_id_null_df = df.filter(df.ticket_id.isNull())
coords_null_df = df.filter(df.coords.isNull())
address_null_df = df.filter(df.address.isNull())

def check_first_null(filtered_df):
    # Check if the filtered DataFrame is empty
    if filtered_df.count() == 0:
        return f"No null values in {filtered_df}."
    else:
        first_row = filtered_df.head()
        return first_row

print(check_first_null(ticket_id_null_df), "\n")
print(check_first_null(coords_null_df), "\n")
print(check_first_null(address_null_df), "\n")

Row(ticket_id=None, type='{ถนน}', organization='สำนักงาน ป.ป.ท.,เขตจอมทอง,ฝ่ายเทศกิจ เขตจอมทอง,ผอ.เขตจอมทอง (นายณัฐพงษ์),กลุ่มกรุงธนเหนือ (รองปลัดฯ เฉลิมพล)', comment=None, photo='https://storage.googleapis.com/traffy_public_bucket/TeamChadChart/corruption_photo2.png', photo_after='https://storage.googleapis.com/traffy_public_bucket/attachment/2022-06/e9596093de70ae8abacd6574f26a2d0f4466fe9f.jpg', coords='100.45568,13.69103', address=None, subdistrict='บางขุนเทียน', district='จอมทอง', province='กรุงเทพมหานคร', timestamp='2022-06-09 23:34:34.98044+00', state='เสร็จสิ้น', star='5', count_reopen=None, last_activity='2022-06-10 11:02:34.607728+00') 

Row(ticket_id='2022-7DABXT', type='{สะพาน}', organization=None, comment='"เคยดีใจมีสายสีน้ำเงินสถานี""แยกไฟฉาย""', photo=None, photo_after=None, coords=None, address=None, subdistrict=None, district=None, province=None, timestamp=None, state=None, star=None, count_reopen=None, last_activity=None) 

Row(ticket_id='2022-7DABXT', type='{สะพาน}', 

From above cells give us the first row with null value from each column selected (ticket_id, coords, address) to tell some relationship of those null value.
1. The ticket_id is null when the state='เสร็จสิ้น', it's mean we cannot drop this column significantly.
2. The address has null value 2 times more than coords. In the first null row we can see both of them are null. So it might tell that if no coords, no address too and not vice versa. We'll check in next step. 

In [9]:
# create new dataframe with only columns we want
df_use = df.select(df['ticket_id'], df['type'], df['coords'], df['address'], df['subdistrict'], df['district'], df['timestamp'])

In [10]:
sub = ['coords', 'address']
df_use = df_use.dropna(how='all', subset=sub)
sub_null_counts = df_use.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in sub])
sub_null_counts.show()

+------+-------+
|coords|address|
+------+-------+
|     1|   2415|
+------+-------+



Due to geolocator.geocode() format need ',' to desperate sub-location, so we need to put in the right format

In [11]:
# example use
# The original data is "1867 จรัญสนิทวงศ์ แขวง บางพลัด เขตบางพลัด กรุงเทพมหานคร 10700 ประเทศไทย".
# The format data is "1867 จรัญสนิทวงศ์, แขวงบางพลัด, เขตบางพลัด, กรุงเทพมหานคร, 10700".
# Note that we can ignore some unneccessary data ex. postal code
# and the format can shuffle data in any place such as this example.  

geolocator = Nominatim(user_agent="ped-hua-som")
location = geolocator.geocode("1867 จรัญสนิทวงศ์, กรุงเทพมหานคร, เขตบางพลัด, แขวงบางพลัด, 10700")
print([location.latitude, location.longitude])
m = folium.Map(location=[location.latitude, location.longitude], zoom_start=15)
folium.Marker(location=[location.latitude, location.longitude]).add_to(m)
m

[13.7885978, 100.5011364]


In [12]:
import re
from pyspark.sql.functions import udf, concat_ws
from pyspark.sql.types import StringType
pattern = re.compile(r'(\d+)\s(.+)\s(\d{5})\s(.+)')

In [13]:
dfp = pd.DataFrame({'Address': ['1867 จรัญสนิทวงศ์ แขวง บางพลัด เขตบางพลัด กรุงเทพมหานคร 10700 ประเทศไทย',
                               '1234 พระราม 9 แขวง ห้วยขวาง เขตห้วยขวาง กรุงเทพมหานคร 10310 ประเทศไทย']})
dfp[['House Number', 'Street', 'Postal Code', 'City']] = dfp['Address'].str.extract(pattern)
dfp['Formatted Address'] = dfp[['House Number', 'Street', 'City', 'Postal Code']].apply(lambda x: ', '.join(x.dropna().values), axis=1)
dfp = dfp.drop('Address', axis=1)
print(dfp)


  House Number                                             Street Postal Code  \
0         1867  จรัญสนิทวงศ์ แขวง บางพลัด เขตบางพลัด กรุงเทพมห...       10700   
1         1234   พระราม 9 แขวง ห้วยขวาง เขตห้วยขวาง กรุงเทพมหานคร       10310   

        City                                  Formatted Address  
0  ประเทศไทย  1867, จรัญสนิทวงศ์ แขวง บางพลัด เขตบางพลัด กรุ...  
1  ประเทศไทย  1234, พระราม 9 แขวง ห้วยขวาง เขตห้วยขวาง กรุงเ...  


In [14]:
# the above flow is wrong 
# 1. assume that street name is next to house number
# 2. find 'แขวง' 'เขต' and others are the same